# Imports

In [8]:
import mne      # toolbox for analyzing and visualizing EEG data
import os       # using operating system dependent functionality (folders)
import pandas   # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import copy     # Can Copy and Deepcopy files so original file is untouched.
from ipywidgets import IntSlider, Output
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

import sys
sys.path.insert(0, 'C:\Projects\ePodium\eegyolk') # path to helper functions
from eegyolk import helper_functions as hf # library useful for eeg and erp data cleaning

# Load data

First the pathways are defined. Those need to be changed in order to run the code. 

In [9]:
# VARIABLES:
path_metadata = "F:\\Stage\\ePODIUM\\Data\\metadata\\"
path_excelFiles = "F:\\Stage\\ePODIUM\\Data\\metadata\\"
path_eeg = "F:\\Stage\\ePODIUM\\Data\\not_zip\\"
path_event = "F:\\Stage\\ePODIUM\\Data\\not_zip\\event_markers"
makeexcelfiles = True # Option to put .txt files into Excel files

Load all meta data files and EEG files

In [10]:
def loadmetadata(filename):
    originalpath = path_metadata + filename + '.txt'    
    if os.path.exists(originalpath):  
        metadata = pandas.read_table(originalpath)
        if(makeexcelfiles):
            excelpath = path_excelFiles + filename + '.xlsx'       
            metadata.to_excel(excelpath)            
        return metadata            
    else: 
        print("PATH NOT FOUND:  " + originalpath) 
        return None

cdi = loadmetadata("cdi")
children = loadmetadata("children")
parents = loadmetadata("parents")

# Process Sound Event Codes file
codes_overview = loadmetadata("CODES_overview")

# cdi
# children
parents
#CODES_overview

,child,emt_mother,klepel_mother,vc_mother,dyslexia_mother_accToMother,emt_father,klepel_father,vc_father,dyslexia_father_accToFather
0,101,105,101,19,Nee,80,64,10,Ja
1,102,116,112,24,Nee,116,94,24,Nee
2,103,90,100,22,Nee,81,65,21,Ja
3,104,81,54,19,Ja,101,77,24,Nee
4,105,101,79,22,Ja,116,101,23,Nee
...,...,...,...,...,...,...,...,...,...
72,173,96,107,23,Nee,?,?,?,?
73,174,?,?,?,?,82,81,18,Nee
74,175,116,108,22,Nee,?,?,?,?
75,176,114,112,21,Nee,?,?,?,?


In [ ]:
eeg = []
eeg_filenames = []
eeg_filelocation = []

for root, dirs, files in os.walk(path_eeg):
    for file in files:
        if file.endswith('.bdf') and len(file) == 8:
            bdffile = mne.io.read_raw_bdf(os.path.join(root, file),preload=True) #preload needs to be true otherwise data is not in memory storage for some mne functions
            eeg.append(bdffile)              
            eeg_filenames.append(file.replace(".bdf", ""))
            eeg_filelocation.append(os.path.join(root,file)) 
            # clear_output(wait=True)            
print(len(eeg), "EEG files loaded")

eeginfo = True
if(eeginfo):
    index = 7
    print('File Location: ',eeg_filelocation[index],'\n')
    print(eeg[index],'\n')
    print(eeg[index].info)

Extracting EDF parameters from F:\Stage\ePODIUM\Data\not_zip\101to110\101to110\101\101a\101a.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3497983  =      0.000 ...  1708.000 secs...
Extracting EDF parameters from F:\Stage\ePODIUM\Data\not_zip\101to110\101to110\101\101b\101b.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3907583  =      0.000 ...  1908.000 secs...
Extracting EDF parameters from F:\Stage\ePODIUM\Data\not_zip\101to110\101to110\102\102a\102a.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3815423  =      0.000 ...  1863.000 secs...
Extracting EDF parameters from F:\Stage\ePODIUM\Data\not_zip\101to110\101to110\102\102b\102b.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3782655  =      0.000 ...  1847.000 secs...
Extracting EDF parameters from F:\St

# Data orientation

Create a widget for index 

In [ ]:
# insert widget for index
index = 8 # EEG data on a single trial of a subject
n_time_samps = eeg[index].n_times
time_secs = eeg[index].times
ch_names = eeg[index].ch_names
n_chan = len(ch_names)  
print('the (cropped) sample data object has {} time samples and {} channels.'
      ''.format(n_time_samps, n_chan))
print('The last time sample is at {} seconds.'.format(time_secs[-1]))
print('The first few channel names are {}.'.format(', '.join(ch_names[:3])))
print()  # insert a blank line in the output

# some examples of raw.info:
print('bad channels:', eeg[index].info['bads'])  # chs marked "bad" during acquisition
print(eeg[index].info['sfreq'], 'Hz')            # sampling frequency
print(eeg[index].info['description'], '\n')      # miscellaneous acquisition info

print(eeg[index].info)

In [ ]:
# extracting data by time index for the first channel
sampling_freq = eeg[index].info['sfreq']
start_stop_seconds = np.array([11, 12])
start_sample, stop_sample = (start_stop_seconds * sampling_freq).astype(int)
channel_index = 6
eeg_selection = eeg[index][channel_index, start_sample:stop_sample]
print(eeg_selection)

In [ ]:
x = eeg_selection[1]
y = eeg_selection[0].T
plt.plot(x, y)

In [ ]:
eeg[index].ch_names

In [ ]:
# extract different channels, create dual axis + widget
channel_names = ['Cz', 'Pz']
two_ch = eeg[index][channel_names, start_sample:stop_sample]
# y_offset = np.array([5e-20,0])  
x = two_ch[1]
y = two_ch[0].T # + y_offset
lines = plt.plot(x, y)
plt.legend(lines, channel_names)


# Events

The event markers are in a seperate .txt file for each child in a separate folder. Run the code below to store all event files in one folder. 

In [ ]:
## SAVE Event Markers to external .txt files:
for i in range(len(eeg)):
      np.savetxt((path_eeg)+"./event_markers\\" + eeg_filenames[i] + ".txt", mne.find_events(eeg[i]), fmt = '%i')
      #clear_output(wait=True)

In [ ]:
#checking if count eeg files == count event files

initial_count = 0
dir = path_event
for path in os.listdir(dir):
    if os.path.isfile(os.path.join(dir, path)):
        initial_count += 1
print(initial_count) # event files

print(len(eeg_filenames)) # eeg files

In [ ]:
events = [] 
events_12 = []
for i in range(len(eeg)):
    events.append(np.loadtxt(path_event+ "\\" + eeg_filenames[i] + ".txt", dtype = int))        
print(len(events), "Event Marker files loaded")

eventinfo = True
participantindex = 5
eventindex = 500
if(eventinfo):
    event_time = events[participantindex][eventindex][0]
    event_ID = events[participantindex][eventindex][2]    
    print("\nParticipant", participantindex,"heard event ID:", event_ID, 
          "after", event_time/eeg[participantindex].info['sfreq'], 'seconds', 
          'as the ' + str(eventindex) + "'th event")    
    print("\nEvent ID:", event_ID, "\nCondition:", codes_overview["condition"][event_ID], "\nStimulus Type:",
         codes_overview["stimulusType"][event_ID], "\nToken nr:", codes_overview["nrOfToken"][event_ID],"\n") 

Creating an event dictionary to map event IDs to trial descriptors

In [ ]:
# M = Multiple / S = Single / T = Tokens            F = First / S = Standard / D = Deviant    
event_dict = {'GiepMT_FS': 1, 'GiepMT_S': 2, 'GiepMT_D': 3,'GiepST_FS': 4, 'GiepST_S': 5, 'GiepST_D': 6,
             'GopMT_FS': 7, 'GopMT_S': 8, 'GopMT_D': 9, 'GopST_FS': 10, 'GopST_S': 11, 'GopST_D': 12}

# Artifacts

In [ ]:
ssp_projectors = eeg[index].info['projs']
eeg[index].del_proj()

In [ ]:
mag_channels = mne.pick_types(eeg[index].info, meg=False, eeg=True, stim=False, eog=False)
print(len(mag_channels))
eeg[index].plot(duration=30, order=mag_channels, n_channels=len(mag_channels),
               )#remove_dc=False)

In [ ]:
fig = eeg[index].plot_psd(tmax=np.inf, fmax=250, average=True)

# Filtering

In order to filter eeg data, the data needs to be filtered, substract baseline, bad channels should be removed and reject artefacts. 

In [ ]:
# check and remove bad channels in all files
def removebadchannel(eeg):
    for i in eeg:
        if len(eeg[index].info['bads']) != 0:
            eeg[index] = mne.pick_types(eeg[index].info, meg=False, eeg=True, exclude='bads')
        

In [ ]:
# use helper functions for bandpass filter
lowpass = widgets.IntText(
    step=0.1,
    description='lowpass:',
    disabled=False
)


highpass = widgets.Text(
    step=0.1,
    description='highpass:',
    disabled=False
)

widgets.VBox([lowpass,highpass])


In [ ]:
# define lowpass and highpass till widgets type are fixed
lowpass = 1
highpass = 40

for i in eeg: eeg[index] = hf.band_pass_filter(eeg[index], lowpass, highpass)

In [ ]:
# plotting filter
filter_params = mne.filter.create_filter(eeg[index].get_data(), eeg[index].info['sfreq'],
                                         l_freq=lowpass, h_freq=highpass)
mne.viz.plot_filter(filter_params, eeg[index].info['sfreq'], flim=(0.01, 5))

In [ ]:
# create notch filter
eeg_picks = mne.pick_types(eeg[index].info, eeg=True)
freqs = (60, 120, 180, 240)

for i in eeg: eeg[index] = eeg[index].notch_filter(freqs=freqs, picks=eeg_picks)




# ERP substraction

In [ ]:
epochs = mne.Epochs(eeg[index], events, tmin=-0.3, tmax=0.7)